In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nvidia.dali as dali
import nvidia.dali.types as types
import nvidia.dali.fn as fn
from nvidia.dali.pipeline import Pipeline
import random

In [14]:
# DALI 파이프라인 설정
batch_size = 1
num_threads = 2
device_id = 0

# 이미지 로드 및 초기 변환 적용
@dali.pipeline_def
def dali_pipeline():
    # 이미지 로드
    jpegs, _ = fn.readers.file(file_root="./augraphy_cache")
    images = fn.decoders.image(jpegs, device="mixed")

    # InkBleed 효과: GaussianBlur로 구현
    blurred_images = fn.gaussian_blur(images, window_size=random.choice([(5, 5), (3, 3)]), sigma=0.4)

    # LightingGradient 효과: 조명 및 밝기 변화
    brightened_images = fn.brightness_contrast(blurred_images, brightness_shift=0.3)

    # OneOf 효과: InkShifter 또는 BleedThrough 유사 구현
    # InkShifter 효과: 텍스트 이동, 블러 및 노이즈 추가
    shifted_images = fn.rotate(brightened_images, angle=random.uniform(18, 23))
    noise_images = fn.noise.salt_and_pepper(shifted_images, prob=random.uniform(0, 0.02))

    # BleedThrough 효과: 이미지 블러 및 색상 변형
    bleed_images = fn.gaussian_blur(noise_images, window_size=(17, 17), sigma=1)

    # Paper Phase 효과: BrightnessTexturize 유사 구현
    textured_images = fn.brightness_contrast(bleed_images, brightness_shift=0.99)

    # Post Phase 효과: Jpeg, Scribbles, Faxify 유사 구현
    # Jpeg 효과: 압축 효과 재현
    # jpeg_images = fn.jpeg_compression(textured_images, quality=15)

    # Scribbles 효과: 임의의 선 추가
    # scribbled_images = fn.drawing.lines(jpeg_images, num_lines=2, line_width=2)

    # Faxify 효과: 스케일, 색상 및 회전 변환
    faxified_images = fn.rotate(textured_images, angle=random.uniform(0, 360))
    faxified_images = fn.gaussian_blur(faxified_images, window_size=(2, 2), sigma=random.uniform(1, 3))

    return faxified_images

In [15]:
# 파이프라인 실행
pipeline = dali_pipeline(batch_size=batch_size, num_threads=num_threads, device_id=device_id)

# 파이프라인 실행
pipeline.build()
outputs = pipeline.run()

RuntimeError: Critical error when building pipeline:
Error in CPU operator `nvidia.dali.fn.readers.file`,
which was used in the pipeline definition with the following traceback:

  File "/tmp/ipykernel_528459/1836649944.py", line 10, in dali_pipeline
    jpegs, _ = fn.readers.file(file_root="./augraphy_cache")

encountered:

Assert on "SizeImpl() > 0" failed: No files found.
C++ context: [/opt/dali/dali/operators/reader/loader/file_label_loader.h:180] 
Current pipeline object is no longer valid.

In [ ]:
"""
# 결과 이미지 저장
import numpy as np
import cv2
for i, output in enumerate(outputs[0].as_cpu().as_array()):
    cv2.imwrite(f"output_{i}.jpg", output)
"""